In [8]:
!wget -O final.csv https://raw.githubusercontent.com/tushar1210/Emotional-Anlaysis-BERT/master/final.csv?token=AFCBJT5FQN5TQ74GCFH2Z2S66B6DW

--2020-06-15 15:09:39--  https://raw.githubusercontent.com/tushar1210/Emotional-Anlaysis-BERT/master/final.csv?token=AFCBJT5FQN5TQ74GCFH2Z2S66B6DW
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 64:ff9b::c7e8:1485, 199.232.20.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|64:ff9b::c7e8:1485|:443... failed: Operation timed out.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.20.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7668756 (7.3M) [text/plain]
Saving to: ‘final.csv’

final.csv           100%[===================>]   7.31M  1.56MB/s    in 5.6s    

2020-06-15 15:11:02 (1.30 MB/s) - ‘final.csv’ saved [7668756/7668756]



In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap



In [10]:
df=pd.read_csv('final.csv')
df.head()

,Unnamed: 0,Country,Tweet,Emotion
0,0,Brazil,un airbnb lejos con piscina y BBQ 🔥,anger
1,1,Brazil,@Airbnb @AirbnbHelp Just got off the phone - I...,anger
2,2,Brazil,@juulianarosa ver no airbnb prima,anger
3,3,Brazil,Vacation real-estate markets are ‘toast’ becau...,anger
4,4,Brazil,Airbnb???jummm 🤔🤔,joy


In [11]:
!pip3 install plotly

     |████████████████████████████████| 11.5MB 486kB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11429 sha256=33ea794501cb068ff86e31f2be08b82c9062e6485eade8c7853134cea331064e
  Stored in directory: /Users/tushar/Library/Caches/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [66]:
df=df.drop(['Unnamed: 0','Tweet'],axis=1)
df.tail()

,Country,Emotion,Code
30,Brazil,anger,BRA
31,Brazil,anger,BRA
32,Brazil,anger,BRA
33,Brazil,sadness,BRA
34,Brazil,joy,BRA
...,...,...,...
217,USA,fear,USA
218,USA,sadness,USA
219,USA,sadness,USA
220,USA,anger,USA


In [32]:
country_dict={}
for c in df.Country:
    country_dict[str(c)]=str(c).upper()[0:3]


Algeria  :  ALG
Argentina  :  ARG
Australia  :  AUS
Austria  :  AUS
Brazil  :  BRA
Canada  :  CAN
Chile  :  CHI
China  :  CHI
Colombia  :  COL
Cuba  :  CUB
Egypt  :  EGY
France  :  FRA
Germany  :  GER
Greece  :  GRE
Hong-kong  :  HON
India  :  IND
Italy  :  ITA
Japan  :  JAP
Macau  :  MAC
Malaysia  :  MAL
Mexico  :  MEX
Morocoo  :  MOR
Morooco  :  MOR
Netherland  :  NET
Peru  :  PER
Portugal  :  POR
Saudi-arabia  :  SAU
SaudiArabia  :  SAU
Singapore  :  SIN
South-Africa  :  SOU
South-Korea  :  SOU
South-korea  :  SOU
Spain  :  SPA
Switzerland  :  SWI
Thailand  :  THA
Tunisia  :  TUN
Turkey  :  TUR
UAE  :  UAE
UK  :  UK
USA  :  USA
Vietnam  :  VIE
Zimbawe  :  ZIM


In [52]:
#add country code
country_dict['Algeria']='DZA'
country_dict['Chile']='CHL'
country_dict['China']='CHN'
country_dict['Germany']='DEU'
country_dict['Greece']='GRC'
country_dict['Hong-kong']='HKG'
country_dict['Japan']='JPN'
country_dict['Malaysia']='MYS'
country_dict['Morocoo']='MAR'
country_dict['Morooco']='MAR'
country_dict['Netherland']='NLD'
country_dict['Portugal']='PRT'
country_dict['Singapore']='SGP'
country_dict['South-Africa']='ZAF'
country_dict['South-korea']='KOR'
country_dict['South-Korea']='KOR'
country_dict['Spain']='ESP'
country_dict['Switzerland']='CHE'
country_dict['UAE']='ARE'
country_dict['UK']='GBR'
country_dict['Zimbawe']='ZWE'
country_dict['Vietnam']='VNM'


for i in sorted(country_dict.keys()):
    print(i," : ", country_dict[i])
    

Algeria  :  DZA
Argentina  :  ARG
Australia  :  AUS
Austria  :  AUS
Brazil  :  BRA
Canada  :  CAN
Chile  :  CHL
China  :  CHN
Colombia  :  COL
Cuba  :  CUB
Egypt  :  EGY
France  :  FRA
Germany  :  DEU
Greece  :  GRC
Hong-kong  :  HKG
India  :  IND
Italy  :  ITA
Japan  :  JPN
Macau  :  MAC
Malaysia  :  MYS
Mexico  :  MEX
Morocoo  :  MAR
Morooco  :  MAR
Netherland  :  NLD
Peru  :  PER
Portugal  :  PRT
Saudi-arabia  :  SAU
SaudiArabia  :  SAU
Singapore  :  SGP
South-Africa  :  ZAF
South-Korea  :  KOR
South-korea  :  KOR
Spain  :  ESP
Switzerland  :  CHE
Thailand  :  THA
Tunisia  :  TUN
Turkey  :  TUR
UAE  :  ARE
UK  :  GBR
USA  :  USA
Vietnam  :  VNM
Zimbawe  :  ZWE


In [56]:
code=[]
for c in df.Country:
    code.append(country_dict[str(c)])    
code[0:10]
df['Code']=code

In [68]:
#anger
anger={}
joy={}
sadness={}
fear={}

for c in code:
    anger[c]=0
    joy[c]=0
    sadness[c]=0
    fear[c]=0
for i in range(len(df.Country)):
    if str(df.Emotion[i])=='anger':
        anger[df.Code[i]]+=1
    if str(df.Emotion[i])=='joy':
        joy[df.Code[i]]+=1
    if str(df.Emotion[i])=='sadness':
        sadness[df.Code[i]]+=1
    if str(df.Emotion[i])=='fear':
        fear[df.Code[i]]+=1

print(anger,'\n',joy,'\n',sadness,'\n',fear)
    

{'BRA': 62, 'USA': 4722, 'DZA': 144, 'ARG': 30, 'AUS': 366, 'CAN': 2856, 'CHL': 13, 'CHN': 47, 'COL': 164, 'CUB': 618, 'EGY': 38, 'FRA': 899, 'DEU': 379, 'GRC': 86, 'HKG': 66, 'IND': 345, 'ITA': 390, 'JPN': 35, 'MAC': 68, 'MYS': 130, 'MEX': 223, 'MAR': 80, 'NLD': 1497, 'PER': 19, 'PRT': 124, 'SAU': 36, 'SGP': 146, 'ZAF': 155, 'KOR': 41, 'ESP': 237, 'CHE': 1089, 'THA': 125, 'TUN': 120, 'TUR': 95, 'ARE': 40, 'GBR': 1548, 'VNM': 84, 'ZWE': 92} 
 {'BRA': 39, 'USA': 3562, 'DZA': 156, 'ARG': 19, 'AUS': 222, 'CAN': 2089, 'CHL': 8, 'CHN': 73, 'COL': 145, 'CUB': 569, 'EGY': 39, 'FRA': 666, 'DEU': 247, 'GRC': 78, 'HKG': 65, 'IND': 456, 'ITA': 332, 'JPN': 114, 'MAC': 65, 'MYS': 124, 'MEX': 167, 'MAR': 79, 'NLD': 1089, 'PER': 10, 'PRT': 107, 'SAU': 54, 'SGP': 132, 'ZAF': 165, 'KOR': 123, 'ESP': 200, 'CHE': 789, 'THA': 149, 'TUN': 128, 'TUR': 55, 'ARE': 47, 'GBR': 1100, 'VNM': 98, 'ZWE': 86} 
 {'BRA': 26, 'USA': 2470, 'DZA': 77, 'ARG': 12, 'AUS': 191, 'CAN': 1757, 'CHL': 3, 'CHN': 32, 'COL': 69, 'C

In [113]:
#create a df
anger_df=pd.DataFrame(anger.items(), columns=['Code', 'Emotion'])
sad_df=pd.DataFrame(sadness.items(), columns=['Code', 'Emotion'])
joy_df=pd.DataFrame(joy.items(), columns=['Code', 'Emotion'])
fear_df=pd.DataFrame(fear.items(), columns=['Code', 'Emotion'])


In [118]:
print(country_dict.keys(),'\n\n\n')
print(list(anger.keys()))

dict_keys(['Brazil', 'USA', 'Algeria', 'Argentina', 'Australia', 'Austria', 'Canada', 'Chile', 'China', 'Colombia', 'Cuba', 'Egypt', 'France', 'Germany', 'Greece', 'Hong-kong', 'India', 'Italy', 'Japan', 'Macau', 'Malaysia', 'Mexico', 'Morocoo', 'Morooco', 'Netherland', 'Peru', 'Portugal', 'Saudi-arabia', 'SaudiArabia', 'Singapore', 'South-Africa', 'South-Korea', 'South-korea', 'Spain', 'Switzerland', 'Thailand', 'Tunisia', 'Turkey', 'UAE', 'UK', 'Vietnam', 'Zimbawe']) 



['BRA', 'USA', 'DZA', 'ARG', 'AUS', 'CAN', 'CHL', 'CHN', 'COL', 'CUB', 'EGY', 'FRA', 'DEU', 'GRC', 'HKG', 'IND', 'ITA', 'JPN', 'MAC', 'MYS', 'MEX', 'MAR', 'NLD', 'PER', 'PRT', 'SAU', 'SGP', 'ZAF', 'KOR', 'ESP', 'CHE', 'THA', 'TUN', 'TUR', 'ARE', 'GBR', 'VNM', 'ZWE']


In [144]:
import plotly.graph_objects as go

def plot(df):
    fig = go.Figure(data=go.Choropleth(
        locations = df['Code'],
        z = df['Emotion'],
        text = df['Code'],
        colorscale = 'Blues',
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_tickprefix = '',
        colorbar_title = 'Emotion',
    ))

    fig.update_layout(
        title_text='Airbnb Tweet Analysis [Fear]',
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
        annotations = [dict(
            x=0.55,
            y=0.1,
            xref='paper',
            yref='paper',
            text='',
            showarrow = False
        )]
    )

    fig.show()

In [139]:
plot(anger_df)

In [141]:
plot(joy_df)

In [143]:
plot(sad_df)

In [145]:
plot(fear_df)